In [1]:
import os

In [2]:
%pwd

'd:\\Deep_Learning_Projects\\Sports_Classification\\notebooks'

In [3]:
os.chdir("../")
%pwd

'd:\\Deep_Learning_Projects\\Sports_Classification'

In [4]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class BaseModelConfig:
    root_dir : Path
    base_model_path : Path
    updated_base_model_path : Path
    image_size_params : list
    learning_rate_params : float
    include_top_params : bool
    weight_params : str
    classes_params : int

In [5]:
from src.Sports_Classification.Constants import *
from src.Sports_Classification.utils.utils import *

logs\02-24-2024-15-33-42.log


In [30]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH
            ):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config["artifacts_root"]])


    def get_base_model(self) -> BaseModelConfig:
        config = self.config['base_model']

        create_directories([self.config['base_model']['root_dir']])

        base_model_config = BaseModelConfig(
            root_dir = Path(config['root_dir']),
            base_model_path = Path(config['base_model_path']),
            updated_base_model_path = Path(config['updated_base_model_path']),
            image_size_params = self.params['IMAGE_SIZE'],
            learning_rate_params = self.params['LEARNING_RATE'],
            include_top_params = self.params['INCLUDE_TOP'],
            weight_params = self.params['WEIGHTS'],
            classes_params = self.params['CLASSES'] 
            )

        return base_model_config

In [31]:
import os, zipfile
from src.Sports_Classification.logger import logging
from src.Sports_Classification.exception import CustomException
import urllib.request as request
from src.Sports_Classification.utils.utils import *
import tensorflow as tf

In [36]:
class PrepareBaseModel:
    def __init__(self, config : BaseModelConfig):
        self.config = config


    def create_base_model(self):
        self.model = tf.keras.applications.vgg16.VGG16(
            input_shape = self.config.image_size_params,
            weights = self.config.weight_params,
            include_top = self.config.include_top_params
        )

        self.save_model(path = self.config.base_model_path, model = self.model)

    @staticmethod
    def save_model(path : Path, model : tf.keras.Model):
        model.save(path)

    
    @staticmethod
    def create_full_model(learning_rate, freeze_all, freeze_till, classes, model):
        if freeze_all is not None:
            for layer in model.layers:
                model.trainable = False
        
        elif (freeze_till is not None) and (freeze_till > 0):
            for layer in model.layers[:freeze_till]:
                model.trainable = False

        flatten_in = tf.keras.layers.Flatten()(model.output)
        prediction = tf.keras.layers.Dense(
                        units=classes,
                        activation="softmax"
                    )(flatten_in)

        full_model = tf.keras.models.Model(
            inputs=model.input,
            outputs=prediction
        )

        full_model.compile(
            optimizer=tf.keras.optimizers.SGD(learning_rate=learning_rate),
            loss=tf.keras.losses.CategoricalCrossentropy(),
            metrics=["accuracy"]
        )

        full_model.summary()
        return full_model
    

    def update_base_model(self):
        self.full_model = self.create_full_model(
            model=self.model,
            classes=self.config.classes_params,
            freeze_all=True,
            freeze_till=None,
            learning_rate=self.config.learning_rate_params
        )

        self.save_model(path=self.config.updated_base_model_path, model=self.full_model)


In [37]:
try:
    config = ConfigurationManager()
    base_model_config = config.get_base_model()
    create_base_model = PrepareBaseModel(config=base_model_config)
    create_base_model.create_base_model()
    create_base_model.update_base_model()
    
except Exception as e:
    raise CustomException(e, sys)

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_11 (InputLayer)       [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0   